# Project 2: Supervised Learning
### Building a Student Intervention System

## 1. Classification vs Regression

Your goal is to identify students who might need early intervention - which type of supervised machine learning problem is this, classification or regression? Why?

**Answer**: This type of machine learning problem is a **classification** since the output we are looking for is a class: in this case, a binary variable "yes" or "no".

## 2. Exploring the Data

Let's go ahead and read in the student dataset first.

_To execute a code cell, click inside it and press **Shift+Enter**._

In [1]:
# Import libraries
import numpy as np
import pandas as pd

In [2]:
# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"
# Note: The last column 'passed' is the target/label, all other are feature columns

Student data read successfully!


Now, can you find out the following facts about the dataset?
- Total number of students
- Number of students who passed
- Number of students who failed
- Graduation rate of the class (%)
- Number of features

_Use the code block below to compute these values. Instructions/steps are marked using **TODO**s._

In [3]:
# TODO: Compute desired values - replace each '?' with an appropriate expression/function call
n_students = student_data.shape[0]
n_features = student_data.shape[1]-1 # Since one of the columns contains the labels
n_passed = student_data[student_data["passed"]=="yes"].shape[0]
n_failed = student_data[student_data["passed"]=="no"].shape[0]
grad_rate = 100.*n_passed / n_students
print "Total number of students: {}".format(n_students)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Number of features: {}".format(n_features)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of students who passed: 265
Number of students who failed: 130
Number of features: 30
Graduation rate of the class: 67.09%


## 3. Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Let's first separate our data into feature and target columns, and see if any features are non-numeric.<br/>
**Note**: For this dataset, the last column (`'passed'`) is the target or label we are trying to predict.

In [4]:
# Extract feature (X) and target (y) columns
feature_cols = list(student_data.columns[:-1])  # all columns but last are features
target_col = student_data.columns[-1]  # last column is the target/label
print "Feature column(s):-\n{}".format(feature_cols)
print "Target column: {}".format(target_col)

X_all = student_data[feature_cols]  # feature values for all students
y_all = student_data[target_col]  # corresponding targets/labels
print "\nFeature values:-"
print X_all.head()  # print the first 5 rows

Feature column(s):-
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']
Target column: passed

Feature values:-
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...    

### Preprocess feature columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation.

In [5]:
# Preprocess feature columns
def preprocess_features(X):
    outX = pd.DataFrame(index=X.index)  # output dataframe, initially empty

    # Check each column
    for col, col_data in X.iteritems():
        # If data type is non-numeric, try to replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])
        # Note: This should change the data type for yes/no columns to int

        # There are a few other binary variables that are not represented by "yes", "no"
        # For this particular dataset, with a relatively small number of features, it is easy
        # to spot them manually:
        if col_data.dtype == object:
            col_data = col_data.replace(['GP', 'MS','U','R','LE3','GT3','T','A'], [0,1,0,1,0,1,0,1])

        # If still non-numeric, convert to one or more dummy variables
        if col_data.dtype == object:
            col_data = pd.get_dummies(col_data, prefix=col)  # e.g. 'school' => 'school_GP', 'school_MS'

        outX = outX.join(col_data)  # collect column(s) in output dataframe

    return outX

X_all = preprocess_features(X_all)
print "Processed feature columns ({}):-\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (44):-
['school', 'sex_F', 'sex_M', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Split data into training and test sets

So far, we have converted all _categorical_ features into numeric values. In this next step, we split the data (both features and corresponding labels) into training and test sets.

In [6]:
# First, decide how many training vs test samples you want
num_all = student_data.shape[0]  # same as len(student_data)
num_train = 300  # about 75% of the data
num_test = num_all - num_train

# TODO: Then, select features (X) and corresponding labels (y) for the training and test sets
# Note: Shuffle the data or randomly select samples to avoid any bias due to ordering in the dataset
from sklearn import cross_validation

X_train, X_test, y_train, y_test = cross_validation.train_test_split(X_all, y_all, 
                                                                     train_size=num_train, random_state=13)
print "Training set: {} samples".format(X_train.shape[0])
print "Test set: {} samples".format(X_test.shape[0])
# Note: If you need a validation set, extract it from within training data

Training set: 300 samples
Test set: 95 samples


## 4. Training and Evaluating Models
Choose 3 supervised learning models that are available in scikit-learn, and appropriate for this problem. For each model:

- What are the general applications of this model? What are its strengths and weaknesses?
- Given what you know about the data so far, why did you choose this model to apply?
- Fit this model to the training data, try to predict labels (for both training and test sets), and measure the F<sub>1</sub> score. Repeat this process with different training set sizes (100, 200, 300), keeping test set constant.

Produce a table showing training time, prediction time, F<sub>1</sub> score on training set and F<sub>1</sub> score on test set, for each training set size.

Note: You need to produce 3 such tables - one for each model.

### Model 1: Support Vector Classification (SVC). 

**Pros**: SVC offers great flexibility since we can use an infinite set of kernel functions. In addition, SVM methods are very memory efficient since, once trained, they only look at a few support vectors, and predictions are very quick. 

**Cons**: However one of its strengths is also one of its weaknesses: the choice of kernel functions being so vast it may be hard to come up with the good one. Sometimes domain knowledge can help us make an informed decission. But other times we may be limited to simply blindly try the standard kernel functions and choose the better performing. Another one of its disadvantages is its long training time. A brute force approach to finding the support vectors calculates the kernel function for every pair of points which can be time expensive.

**Why?**: We have a relatively modest dataset in terms of number of observation so we shouldn't turn away from SVC because of training cost. We have also split our factor variables into multiple dimensions and SVM tends to do well in these situations. Given the relative high number of features and the relatively small number of observations, SVC should be able to learn well from our data.

In [7]:
# Train a model
import time

def train_classifier(clf, X_train, y_train):
    print "Training {}...".format(clf.__class__.__name__)
    start = time.time()
    clf.fit(X_train, y_train)
    end = time.time()
    print "Done!\nTraining time (secs): {:.3f}".format(end - start)

# TODO: Choose a model, import it and instantiate an object
from sklearn import svm
clf1 = svm.SVC()

# Fit model to training data
train_classifier(clf1, X_train, y_train)  # note: using entire training set here
print clf1  # you can inspect the learned model by printing it


Training SVC...
Done!
Training time (secs): 0.009
SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)


In [8]:
# Predict on training set and compute F1 score
from sklearn.metrics import f1_score

def predict_labels(clf, features, target):
    print "Predicting labels using {}...".format(clf.__class__.__name__)
    start = time.time()
    y_pred = clf.predict(features)
    end = time.time()
    print "Done!\nPrediction time (secs): {:.3f}".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')

train_f1_score = predict_labels(clf1, X_train, y_train)
print "F1 score for training set: {}".format(train_f1_score)

Predicting labels using SVC...
Done!
Prediction time (secs): 0.008
F1 score for training set: 0.887417218543


In [9]:
# Predict on test data
print "F1 score for test set: {}".format(predict_labels(clf1, X_test, y_test))

Predicting labels using SVC...
Done!
Prediction time (secs): 0.005
F1 score for test set: 0.791946308725


In [10]:
# Train and predict using different training set sizes
def train_predict(clf, X_all, y_all, train_sizes, test_size):
    for i in train_sizes:
        X_train, X_test, y_train, y_test = cross_validation.train_test_split(X_all, y_all, train_size= i, 
                                                                             test_size=test_size, random_state=66)
        print "------------------------------------------"
        print "Training set size: {}".format(len(X_train))
        train_classifier(clf, X_train, y_train)
        print "F1 score for training set: {}".format(predict_labels(clf, X_train, y_train))
        print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test))

# TODO: Run the helper function above for desired subsets of training data
# Note: Keep the test set constant
train_size = [100, 200, 300]
test_size = 95

train_predict(clf1, X_all, y_all, train_size, test_size)

------------------------------------------
Training set size: 100
Training SVC...
Done!
Training time (secs): 0.008
Predicting labels using SVC...
Done!
Prediction time (secs): 0.001
F1 score for training set: 0.909090909091
Predicting labels using SVC...
Done!
Prediction time (secs): 0.001
F1 score for test set: 0.787096774194
------------------------------------------
Training set size: 200
Training SVC...
Done!
Training time (secs): 0.013
Predicting labels using SVC...
Done!
Prediction time (secs): 0.007
F1 score for training set: 0.895833333333
Predicting labels using SVC...
Done!
Prediction time (secs): 0.003
F1 score for test set: 0.772413793103
------------------------------------------
Training set size: 300
Training SVC...
Done!
Training time (secs): 0.011
Predicting labels using SVC...
Done!
Prediction time (secs): 0.006
F1 score for training set: 0.872807017544
Predicting labels using SVC...
Done!
Prediction time (secs): 0.002
F1 score for test set: 0.786666666667


### Model 2: Gaussian Naive Bayes

**Pros**: Gaussian Naive Bayes (GNB) is a simple and robust model that relies on Bayes' conditional probability rules and assumes conditional independance between the different variables. As such it is a simple model and a very quick learner. Even with the massive assumption of conditional independance, this model has been shown to work quite well in a variety of scenarios.

**Cons**: As with the previous model, one of its strengths is also one of its weaknesses. The assumption of conditional independence may not be correct for the problem at hand and by ignoring any possible relation between variables it may be hard to capture the nature of our data. In addition, the model offers no parameters to tweak, making it at the same time very straight-forward to use but with little flexibility.

**Why?**: Given my limited knowledge of the subject under study, I am in no position to judge if the assumptions of this model are correct. But since it is not a very complex (in time and memory), it might give us a different baseline to compare other models too (it may even outperform them!).

In [11]:
# TODO: Train and predict using two other models
from sklearn import naive_bayes
clf2 = naive_bayes.GaussianNB()
train_classifier(clf2, X_train, y_train)
train_predict(clf2, X_all, y_all, train_size, test_size)

Training GaussianNB...
Done!
Training time (secs): 0.006
------------------------------------------
Training set size: 100
Training GaussianNB...
Done!
Training time (secs): 0.001
Predicting labels using GaussianNB...
Done!
Prediction time (secs): 0.000
F1 score for training set: 0.866242038217
Predicting labels using GaussianNB...
Done!
Prediction time (secs): 0.001
F1 score for test set: 0.739130434783
------------------------------------------
Training set size: 200
Training GaussianNB...
Done!
Training time (secs): 0.003
Predicting labels using GaussianNB...
Done!
Prediction time (secs): 0.002
F1 score for training set: 0.825174825175
Predicting labels using GaussianNB...
Done!
Prediction time (secs): 0.001
F1 score for test set: 0.768115942029
------------------------------------------
Training set size: 300
Training GaussianNB...
Done!
Training time (secs): 0.003
Predicting labels using GaussianNB...
Done!
Prediction time (secs): 0.001
F1 score for training set: 0.814988290398
Pr

### Model 3: K-Nearest Neighbors

**Pros**: The K-Nearest Neighbors (KNN) method is also very simple and robust. It has the advantage of being a lazy method so when we train the model with the three different training set sizes, it does so instantly. Another one of its advantages is the fairly intuitive K parameter, so when optimizing it (via CV or some other method) we can readily asses whether the result makes some sense or not.

**Cons**: In KNN we rely on some metric to define distance. This function can be the standard Euclidean distance or some other metric. Domaine knowledge can help make an educated guess as to which "distance" function to use, but otherwise, the use of the standard Euclidean distance can seriously limit the model predictions. Another problem we may encounter with KNN is that it is very expensive in space since we need to store all our training points. 

**Why?**: In this particular case we are do not run into storage problems, since the dataset is relatively small. In addition since we want to train the model with three different training set sizes, KNN is the quickest way since we don't actually have to train the model and we can add more points to the training set without worrying about having to retrain it.

In [16]:
# TODO: Train and predict using two other models
from sklearn import neighbors
clf3 = neighbors.KNeighborsClassifier()
train_classifier(clf3, X_train, y_train)
train_predict(clf3, X_all, y_all, train_size, test_size)

Training KNeighborsClassifier...
Done!
Training time (secs): 0.002
------------------------------------------
Training set size: 100
Training KNeighborsClassifier...
Done!
Training time (secs): 0.001
Predicting labels using KNeighborsClassifier...
Done!
Prediction time (secs): 0.002
F1 score for training set: 0.883116883117
Predicting labels using KNeighborsClassifier...
Done!
Prediction time (secs): 0.001
F1 score for test set: 0.789473684211
------------------------------------------
Training set size: 200
Training KNeighborsClassifier...
Done!
Training time (secs): 0.001
Predicting labels using KNeighborsClassifier...
Done!
Prediction time (secs): 0.005
F1 score for training set: 0.85
Predicting labels using KNeighborsClassifier...
Done!
Prediction time (secs): 0.002
F1 score for test set: 0.785714285714
------------------------------------------
Training set size: 300
Training KNeighborsClassifier...
Done!
Training time (secs): 0.002
Predicting labels using KNeighborsClassifier...


**Recap**: We present a recap table of the training and predicting times as well as the F1 score for the test set for the three different training set sizes:

In [17]:
data = np.array([[0.002,0.004,0.008,0.001,0.001,0.001,0.002,0.001,0.001],
                 [0.002,0.002,0.001,0.001,0.000,0.000,0.004,0.001,0.002],
                 [0.909090909091,0.895833333333,0.872807017544,
                  0.866242038217,0.825174825175,0.814988290398,
                  0.883116883117,0.85,0.860411899314],
                 [0.787096774194,0.772413793103,0.786666666667,
                  0.739130434783,0.768115942029,0.721804511278,
                  0.789473684211,0.785714285714,0.765957446809]])
method = ["SVC 100", "SVC 200", "SVC 300", "GNB 100", "GNB 200", "GNB 300", "KNN 100", "KNN 200", "KNN 300"]
feats = ["Train t", "Pred t", "Train F1","Test F1"]

print pd.DataFrame(data, feats, method)

           SVC 100   SVC 200   SVC 300   GNB 100   GNB 200   GNB 300  \
Train t   0.002000  0.004000  0.008000  0.001000  0.001000  0.001000   
Pred t    0.002000  0.002000  0.001000  0.001000  0.000000  0.000000   
Train F1  0.909091  0.895833  0.872807  0.866242  0.825175  0.814988   
Test F1   0.787097  0.772414  0.786667  0.739130  0.768116  0.721805   

           KNN 100   KNN 200   KNN 300  
Train t   0.002000  0.001000  0.001000  
Pred t    0.004000  0.001000  0.002000  
Train F1  0.883117  0.850000  0.860412  
Test F1   0.789474  0.785714  0.765957  


## 5. Choosing the Best Model

- Based on the experiments you performed earlier, in 1-2 paragraphs explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?
- In 1-2 paragraphs explain to the board of supervisors in layman's terms how the final model chosen is supposed to work (for example if you chose a Decision Tree or Support Vector Machine, how does it make a prediction).
- Fine-tune the model. Use Gridsearch with at least one important parameter tuned and with at least 3 settings. Use the entire training set for this.
- What is the model's final F<sub>1</sub> score?

From the table above we can already draw a few conclusions and confirm some of our assumptions. First, we notice the expected and general trend that as the training set size increases, the error in the training set increases (the F1 score decreases). This is because with fewer points in the training set, the model tends to overfit it more, making the train error small but not generalizing well. The training and predicting times also agree with our intuition. The most expensive model is the SVC, as can be seen from its testing time. Both GNB and KNN have very small training times but for very different reasons: for GNB it steams from the model simplicity and assumptions, whereas for KNN it is because KNN is lazy. 

The results that are bit more difficult to interpret are the Test F1 scores. Let's look at the three models one by one. Starting with the **SVC** we see that actually we obtain the highest test F1 score when there are only 100 points in the training set. However, as it drops more than a point when increasing from 100 test points to 200, the trend is reversed when going from 200 to 300, increasing to almost the same level as before. 

For **GNB** we see the expected increase in test F1 score as we increase the number of training points, but then it drops sharply when n=300.

Finally, for **KNN** we also see the best score when n=100, and it goes down as the number of points goes up.

I believe these are inconsistencies due to limited size of our dataset and the unbalanced nature of the labels (67% pass). We have a very limited dataset, under 400 ovservations for 44 features. In addition to that, we don't have a balanced number of "pass" and "fail". This can be a problem since, specially when the training set has 100 points and we have kept the test set fixed at 95 points, because of the random sampling we may find a case when most "fail" cases are lost, not belonging to the test nor the training set. We have verified this by changing the random seed in our train-test split. Changing the random seed alters dramatically the results. 

Because of the high number of features and the somehow limited number of points in our training set, and because its "out of the box" performance is pretty good, we will concentrate in improving the SVC classifier. 

### Support Vector Machines:

In order to understand SVM let's start with a simple example: say you want to predict whether a particular athlete is a basketball or a baseball player purely based on height. You are giving a list of 500 basketball players' heights and another list of 500 baseball players' heights. When looking at these data you find that **all** baseball players are below 1,75m and **all** basketball players are above 1,95m. Now, in order to make our predictions we have many many options of where to set the cutoff: we could say, for example, that all athletes taller than 1,75m are basketball players. Our we could say that all athletes above 1,94m are basketball players. Where do we draw the line?

In order to answer this question we have to look at the concept of margin. We'll continue with our example. Say that we draw the line at 1,80m. What is the margin? The "distance" from the tallest baseball player (1,75m) to our cutoff height is 5cm, and from the cutoff to the shortest basketball player (1,95m) is 15cm. SVM tries to **maximize** this margin, that is, it tries to find the cutoff height such that the distance to the different classes is maximum. We can see that in our case that would mean setting the mark at 1,85m since then the distance to the tallest baseball player and to the shortest basketball player is maximum (10cm). What is really nice about this method is that, for this case, only the talles baseball player and the shortest basketball player are needed. We do not need the rest of the data, we can maximize the margin using only these two data points. We call these datapoints the **support vectors**.

The example we looked at was a simple one-dimensional problem, but we can easily see how it generalizes to higher dimensions. If insted of looking at just height we looked at height and weight, we would have to draw our data points in an x-y graph, and we would have a line dividing the two different classes. If we added a third dimension we would have a plane split the classes and in higher dimensions we would have hyperplanes.

We will use Sklearn's grid_search to estimate two parameters:
* The kernel type: linear or RBF.
* The C parameter which specifies how much to penalise misclassifications. A low C value will try to make the surface smooth, whereas a high C value tries to correctly classify as many training points as possible.
* The class_weight parameter. By default, the model gives every point an equal weight of 1. By setting it to "balanced", 
* The gamma parameter. In RBF, the measure of similarity between two points takes the form $d(x,x') = exp \left(-\dfrac{||x-x'||^{2}}{2\sigma^{2}}\right)$ which can also be expressed as $d(x,x') = exp(-\gamma ||x-x'||^{2})$ by setting $\gamma = \dfrac{1}{2\sigma^2}$. We see that it measures how "fast" the similarity drops from 1 when $x=x'$ to 0.

In our grid_search we will go beyond the standard three-fold CV and try a five-fold cross-validation. We will also use the F1 score as the scoring function. However we cannot use it as is, since it cannot identify the positive label and we cannot pass any arguments. So we'll make a scorer using make_scorer

In [36]:
# TODO: Fine-tune your model and report the best F1 score
from sklearn import grid_search
from sklearn import metrics
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X_all, y_all, 
                                                                     train_size=num_train, random_state=23)
parameters = {'kernel':('linear','rbf'), 
              'C':np.arange(1,5,1), 
              'class_weight':(None,'balanced'),
              'gamma':np.logspace(-3,2,num=12)
}

custom_f1_scorer = metrics.make_scorer(f1_score, pos_label='yes')

clf4 = grid_search.GridSearchCV(svm.SVC(),parameters,cv=5,scoring=custom_f1_scorer)
clf4.fit(X_train,y_train)
print clf4.best_params_

{'kernel': 'rbf', 'C': 2, 'gamma': 0.0081113083078968723, 'class_weight': None}


In [37]:
y_pred = clf4.predict(X_test)
print "Test score without grid search: {}".format(f1_score(y_test, svm.SVC().fit(X_train, y_train).predict(X_test), pos_label='yes'))
print "Test score with grid search: {}".format(f1_score(y_test, clf4.predict(X_test), pos_label='yes'))

Test score without grid search: 0.805369127517
Test score with grid search: 0.813333333333


We see that the test score improves after using grid search to find the optimal value of the parameters. However, once again, due to the relatively small amount of data this dataset consists of, the random seed can have a serious impact in the test score. Depending on the random seed we use we might get lucky with our testing set and see more or less (or even negative! set, for example, the random_state=18) improvement with the grid search.

---------------------------------------

Thanks to Udacity's review, we see that there is an alternative train-test split we can use in the cases where we have more events belonging to one class than the other. Here we will quickly explore how it alters the last results above

In [48]:
from sklearn.cross_validation import StratifiedShuffleSplit
strat_shuffle_split = StratifiedShuffleSplit(y_all, test_size=0.3, random_state=2)
for train_index, test_index in strat_shuffle_split:
    X_train, X_test = X_all.iloc[train_index], X_all.iloc[test_index]
    y_train, y_test = y_all[train_index], y_all[test_index]

clf4.fit(X_train,y_train)
print clf4.best_params_

{'kernel': 'rbf', 'C': 4, 'gamma': 0.0028480358684358021, 'class_weight': None}


In [49]:
y_pred = clf4.predict(X_test)
print "Test score without grid search: {}".format(f1_score(y_test, svm.SVC().fit(X_train, y_train).predict(X_test), pos_label='yes'))
print "Test score with grid search: {}".format(f1_score(y_test, clf4.predict(X_test), pos_label='yes'))

Test score without grid search: 0.806282722513
Test score with grid search: 0.8125


There definitely is a difference between using a simple train-test-split and a stratified shuffle split. The values are more uniform, and don't vary as wildely when using a different random seed. However there are still a few cases (random seeds 10 and 43 for example) where we get worse performance when using grid search than without, and the overall score is not very good. We conclude that although a stratified shuffle split helps in achieving more reliable results, our dataset may be a little too small, since we still get different results based on the random seed used.